In [8]:
#import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

### 1. Formulate the statement of the interpolation problem with Cubic Spline [mathematical formula]

We have a mesh function ${y_i = f(x)}$ defined on the segment $[a; b]$ on mesh $\Omega_n$, where $n$ - number of known points.

We need to fill the gaps between known points with function $S_m(x)$, where $m$ - polynomial degree (number of partial segments) using piecewise-global method.
For our needs we can use different polynomial for each gap between known points in form:
$$S_{3,i}=a_{0,i}+a_{1,i}(x-x_i)+a_{2,i}(x-x_i)^2+a_{3,i}(x-x_i)^3$$
Since total number of partial segments is equal to m, so overall we get $4m$ unknowns.

Now we can union all polynomials and get a single cubic curve on the segment $[a; b]$. 
But the problem is that two adjacent polynomials didn't know anything about each other.
    To solve it we need to ensure the continuity of the entire spline, for this we introduce $stiching$ $conditions$. For some point $x_i$, that is common for the segment $i-1$, which belongs to partial segment $[x_i, x_{i+1}]$, and for segment $i$ which belongs to partial segment $[x_i, x_{i+1}]$, this condition is in form: $$S_{3, i-1}^{(p)}(x)|_{x=x_i} = S_{3, i}^{(p)}(x)|x=x_i$$
        The number of inner knots is $m-1$, so we have $3(m-1)$ conditions (equations) for the desired coefficients. So, the total number of equations is equal to $4m-2$. To find other two conditions, (simple case) we can set to zero derivatives at the ends of the segment: $$m_0=0, m_n = 0$$
        If all conditions are satisfied, so our resulting polynomial is a function of class $C^2[a;b]$ (has the second continious derivative on the segment $[a,b]$)

### 2. Formulate the functional and differential compatibility conditions [mathematical formula]

1) Functional compability condition is:
$$\delta\tilde{f_m}(x_i,\bar{a}) = \tilde{f_m}(x_i,\bar{a})-f(x_i) = S_{3,i}(x_i) - f(x_i) = 0$$

2) Differential compability condition, in its turn, is:
$$\delta\tilde{f_m}^{(p_1)}(x_i,\bar{a}) = \tilde{f_m}^{(p_1)}(x_i,\bar{a})-f^{(p_1)}(x_i) = S''_{3,i}(x_i)-f''(x_i) = 0$$
where $p_1$ between $0$ and $r$ (smoothness of the resulting function (maximum order of derivative that is continuous on the segment $[a;b]$))

### 3. Formulate stitching conditions [mathematical formula]

Stiching conditions of partial segments $S_{m,i}$: $$S_{m,i-1}^{(p_2)}(x)|_{x=x_i} = S_{m,i}^{(p_2)}(x)|_{x=x_i} , i = \overline{1,n-1}$$
where $p_2$ is integer, between $0$ and $r$, but none of possible values of $p_2$ not equal to any of $p_1$ values








### 4. Justify why these conditions provide you with the required smoothness [thesis text, no more than 500 characters]

Stiching conditions paired with compability conditions provides continuity of $S_m^{(p)}(x)$, $p = 0,1,...r$, in all inner points $x_i$, which define partial segment stiching points, i.e. $(x_i,S_m(x_i))$, $(i=\overline{1,n-1})$. This guarantees that the condition is met: $S_m(x)\in C_r[a,b]$. Otherwords, stiching conditions, provides the resulting interpolant function to have the same derivatives on the joint knots, which is the condition for the continuity of function. On the other side, functional and differential compatibility conditions fulfill smoothness on the inside knots, by following smoothness of resulting interpolated function.

### 5. Derive dependency formula: the dependence of the second derivatives at the grid nodes on the increment of the function (the function values difference on the grid nodes). [Mathematical formulas derivation. Detailed, with clear transitions]

$$S_{3,i}(x)=a_{0,i}+a_{1,i}(x-x_i)+a_{2,i}(x-x_i)^2+a_{3,i}(x-x_i)^3$$
Second derivitive:
$$S''_{3,i}(x)=2a_{2,i}+6a_{3,i}x-6a_{3,i}x_i$$
$$h_{i+1}=x_{i+1}-x_i, \Delta f_i=f_{i+1}-f_i, m_i=f''(x_i), \Delta m_i=m_{i+1}-m_i$$

From functional conditions (eq 1 from ex. 2):

$S_{3,i}(x_i)-f(x_i)=0$, so $a_{0,i}=f(x_i)=f_i$

From differential compability conditions for $p = {0,2}$ (eq 2 from ex. 2):

$2a_{2,i}+6a_{3,i}x-6a_{3,i}x_i = f''(x_i)$

$2a_{2,i}=m_i$

$a_{2,i}=\frac{m_i}{2}$

$S_{3,i}(x_i)-f(x_i)=0$, so $a_{0,i}=f(x_i)=f_i$

To $S''_{3,i}(x_{i+1})-f''(x_i)=0$ substitute $a_{2,i}$):

$f''(x_{i+1}) = m_i+6a_{3+i}x_{i+1}-6a_{3,i}x_i$

$m_i+6a_{3,i}h_{i+1}=m_{i+1}$

$a_{3,i}=\frac{ \Delta m_i}{6h_{i+1}}$

To $S_{3,i}(x_{i+1})-f(x_{i+1}) = 0$ substitute $a_{0,i}, a_{2,i}, a_{3,i}$

$f(x_{i+1})=a_{0,i}+a_{1,i}(x_{i+1}-x+i)+a_{2,i}(x_{i+1}-x_i)^2+a_{3,i}(x_{i+1}-x_i)^3$

$a_{1,i}=\frac{\Delta f_i}{h_{i+1}} - \frac{m_ih_{i+1}}{2} \frac{\Delta m_ih_{i+1}}{6}$

$S_{3,i}=S_{3,i-1}$:

$S'_{3,i}(x)=a_{i,1}+2a_{2,i}(x-x_i)+3a_{3,i}(x-x_i)^2$

$S'_{3,i-1}(x)=a_{1,i-1}+2a_{2,i-1}(x-x_{i-1})+3a_{3,i-1}(x-x_{i-1})^2$

$S'_{3,i}(x)|_{x=x_i}=S'_{3,i-1}(x)|_{x=x_i}$

Thus, $a_{1,i} = a_{1,i-1}+2a_{2,i-1}h_i+3a_{3,i-1}h_i^2$

Consider $x = x_i$, since derivatives intersect in one point($S_{3,i}(x_i)=S_{3,i-1}(x_i)$):

$\frac{\Delta f_i}{h_{i+1}} - \frac{m_in_{i+1}}{3} - \frac{m_{i+1}n_{i+1}}{6} = 
\frac{\Delta f_{i-1}}{h_i} - \frac{m_{i-1}n_i}{3} - \frac{m_in_i}{6} + m_{i-1}h_i + \frac{m_ih_i}{2} - \frac{m_{i-1}h_i}{2}$

Thus,
$\frac{h_i}{6}m_{i-1} + \frac{h_i+h_{i+1}}{3}m_i + \frac{h_{i+1}}{6}m_{i+1} =
 \frac{\Delta f_i}{h_{i+1}} - \frac{\Delta f_{i-1}}{h_i}$, $i=\overline{1,n-1}$

### 6. Create a system of equations using this formula [Matrix representation. Mathematical formulas]

$\begin{vmatrix} 
1 & 0 & 0 & 0 \\
1 & (x_{i+1}-x_i) & (x_{i+1}-x_{i})^2 & (x_{i+1}-x_i)^3 \\
0 & 0 & 2 & 0 \\
0 & 0 & 2 & 6(x_{i+1}-x_{i})
\end{vmatrix} \cdot \begin{vmatrix} 
a_{0,i} \\
a_{1,i} \\
a_{2,i} \\
a_{3,i} \\
\end{vmatrix} = \begin{vmatrix} 
f_i \\
f_{i+1} \\
m_i \\
m_{i+1}
\end{vmatrix}$

$\frac{1}{6} \begin{vmatrix} 
h_1 & 2(h_1 + h_2) & h_2 & 0 & 0 & ... \\
0 & h_2 & 2(h_2 + h_3) & h_3 & 0 & ... \\
... & ... & ... & ... & ... & ... \\
... & 0 & 0 & h_n-1 & 2(h_n-1 + h_n) & h_n
\end{vmatrix} \cdot \begin{vmatrix} 
m_0 \\
m_1 \\
m_2 \\
... \\
m_n
\end{vmatrix} = \begin{vmatrix} 
\frac{\Delta f_1}{h_2} - \frac{\Delta f_0}{h_1} \\
\frac{\Delta f_2}{h_3} - \frac{\Delta f_1}{h_2} \\
... \\
\frac{\Delta f_{n-1}}{h_n} - \frac{\Delta f_{n-2}}{h_{n-1}}
\end{vmatrix}$









### 7. Explain what is an unknown variable in this system. whether the system is closed with respect to an unknown variable. What is missing for closure. [Text, no more than 200 characters]

System is not closed relatevly ${m_i(i=\overline{1,n-1})}$ (unknown variable). It's needed to appoximate value of $m_i$ on the boundaries (close the system).
1) Simple way - we can set $m_0 = 0; m_n = 0$ (natural spline conditions).

2) Proportion and two three-point equations allows to close the system:

$\frac{\Delta m_{k-1}}{h_k} = \frac{\Delta m_k}{h_{k+1}} (k=\overline{1,n-1})$

$\frac{m_0}{h_1}-(\frac{1}{h_1} + \frac{1}{h_2})m_1+\frac{m_2}{h_2} = 0$

$\frac{m_{n-2}}{h_{n-1}}-(\frac{1}{h_{n-1}}+\frac{1}{h_n})m_{n-1}+\frac{m_n}{h_n} = 0$



### 8. Bring this matrix to the appropriate form to use the Tridiagonal matrix algorithm [Mathematical derivation. Use Gauss Elimination]

$\begin{vmatrix}
-\frac{h_1 + h_{2}}{3} & \frac{h_{2}}{6} & 0 & 0 & 0 & ... \\
\frac{h_2}{6} & -\frac{h_2 + h_{3}}{3} & \frac{h_{3}}{6} & 0 & 0 & ... \\
... & ... & ... & ... & ... & ... \\
... & 0 & 0 & 0 & \frac{h_n-1}{6} & -\frac{h_n-1 + h_{n}}{3}
\end{vmatrix} \cdot \begin{vmatrix} 
m_0 \\
m_1 \\
m_2 \\
... \\
m_n
\end{vmatrix} = \begin{vmatrix} 
\frac{\Delta f_1}{h_2} - \frac{\Delta f_0}{h_1} \\
\frac{\Delta f_2}{h_3} - \frac{\Delta f_1}{h_2} \\
... \\
\frac{\Delta f_{n-1}}{h_n} - \frac{\Delta f_{n-2}}{h_{n-1}}
\end{vmatrix}$

1) Divide first row by $-\beta=(\frac{h_1+h_2}{3})$

2) Subtruct 1st row multiplied by $\alpha_2=(\frac{h_2}{6})$ from 2nd row

3) Divide 2nd row by $(-\beta_2+\frac{\gamma_1\alpha_2}{\beta_1}) = (\frac{-h_2^2+4(h_1+h_2)(h_2+h_3)}{12(h_1+h_2)})$

After performing this steps on other rows, we have matrix in following form:

$\begin{vmatrix} 
1 & -P_1 & 0 & 0 & ... & Q_1 \\
0 & 1 & -P_2 & 0 & ... & Q_2 \\
0 & 0 & 1 & -P_3 & ... & Q_3\\
... & ... & ... & ... & ... & ... \\
... & ... & ... & 1 & -P_{n-1} & Q_{n-1}
\end{vmatrix}$

where $P_i$ and $Q_i$ are derived in question 9.





### 9. Derive formulas of direct pass and reverse pass of Tridiagonal matrix algorithm [Mathematical formals]

$m_i = P_i m_{i+1} + Q_i$, $i=\overline{1, n-2}$

$m_{i-1} = P_{i-1}m_i + Q_{i-1}$

We can substitute in general equation of the system:

$\frac{(P_{i-1}m_i + Q_{i-1}) h_i}{6} - \frac{m_i(h_i+h_{i+1})}{3} + \frac{m_{i+1}h_{i+1}}{6} = \frac{\Delta f_{i}}{h_{i+1}} - \frac{\Delta f_{i-1}}{h_{i}}$

$\alpha_i = \frac{h_i}{6}, \beta_i = \frac{h_i + h_{i+1}}{3}, \gamma_i = \frac{h_{i+1}}{6}, \delta_i = \frac{\Delta f_{i}}{h_{i+1}} - \frac{\Delta f_{i-1}}{h_{i}}$

$\alpha_i (P_{i-1}m_i + Q_{i-1}) - \beta_i m_i + \gamma_i m_{i+1} = \delta_i \\
\alpha_i P_{i-1}m_i + \alpha_i Q_{i-1} - \beta_i m_i + \gamma_i m_{i+1} = \delta_i \\
m_i(\alpha_i P_{i-1} - \beta_i) + \alpha_i Q_{i-1} + \gamma_i m_{i+1} = \delta_i \\
m_i = -\frac{\gamma_i m_{i+1}}{\alpha_i P_{i-1} - \beta_i} - \frac{\alpha_i Q_{i-1}}{\alpha_i P_{i-1} - \beta_i} + \frac{\delta_i}{\alpha_i P_{i-1} - \beta_i} = \frac{\gamma_i m_{i+1}}{\beta_i - \alpha_i P_{i-1}} + \frac{\alpha_i Q_{i-1} - \delta_i}{\beta_i - \alpha_i P_{i-1}}$

Thus, forward pass:
$P_i = \frac{\gamma_i}{\beta_i-\alpha P_{i-1}}, P_1=\frac{\gamma_1}{\beta_1}$

$Q_i = \frac{\alpha_iQ_{i-1}-\delta_i}{\beta_i-\alpha P_{i-1}}, Q_1=\frac{\delta_1}{\beta_1}$

Last equation:

$m_{n-1} = P_{n-1}m_n + Q_{n-1} \\
\alpha_n m_{n-1} - \beta_n m_n = \delta_n$

Substitute $m_{n-1}$:

$\alpha_n(P_{n-1}m_n + Q_{n-1}) - \beta_n m_n = \delta_n \\
\alpha_n P_{n-1}m_n + \alpha_n Q_{n-1} - \beta_n m_n = \delta_n \\
m_n(\alpha_n P_{n-1} - \beta_n) = \delta_n - \alpha_n Q_{n-1} \\
m_n = \frac{\delta_n - \alpha_n Q_{n-1}}{\alpha_n P_{n-1} - \beta_n} = \frac{\alpha_n Q_{n-1} - \delta_n}{\beta_n - \alpha_n P_{n-1}}$

Thus, backward pass:

$x_n=\frac{\alpha_nQ_{n-1}-\delta_n}{\beta_n-\alpha_nP_{n-1}} = Q_n$

$x_i=P_ix_{i+1}+Q_i, i = \overline{1, n-1}$





### 10. Implement code prototype of the future algorithm implementation. Classes/methods (if you use OOP), functions. The final implementation (on language chosen by you) should not differ from the functions declared in the prototype. [Python code]

In [ ]:
def estimate_parameters(i, x, y):
    # returns a, b, c, q
    return x[i] / 6 - x[i - 1] / 6, -x[i + 1] / 3 + x[i - 1] / 3, x[i + 1] / 6 - x[i] / 6, (
            (y[i + 1] - y[i]) / (x[i + 1] - x[i])) - ((y[i] - y[i - 1]) / (x[i] - x[i - 1]))

def forward_backward(x, y, n):
    P = [0]
    Q = [0]
    m = [0] * n
    # formward pass
    for i in range(1, n - 1):
        a, b, c, q = estimate_parameters(i, x, y)

        if i == 1:
            P.append(c / b)
            Q.append(- q / b)
            continue

        P.append(c / (b - a * P[i - 1]))
        Q.append((a * Q[i - 1] - q) / (b - a * P[i - 1]))
    # backward pass
    m[n - 2] = Q[n - 2]
    for i in range(n - 2, 1, -1):
        m[i - 1] = P[i - 1] * m[i] + Q[i - 1]
    return m

def result(test_point, m, x, y, n):
    for i in range(n):
        if test_point < x[i]:
            iter = i - 1
            difference = (test_point - x[i])
            a_0, a_1, a_2, a_3 = y[iter], ((y[iter + 1] - y[iter]) / (x[iter + 1] - x[iter])) - ((m[iter] * (x[iter + 1] - x[iter])) / 2) - (
                    ((m[iter + 1] - m[iter]) * (x[iter + 1] - x[iter])) / 6), m[iter] / 2, (m[iter + 1] - m[iter]) / (6 * (x[iter + 1] - x[iter]))
            return a_0 + a_1 * x_dif + a_2 * difference ** 2 + a_3 * difference ** 3

# [a; b] = [0; 0.9]
x = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
test_x = [0.05, 0.15, 0.25, 0.35, 0.45, 0.55, 0.65, 0.75, 0.85]
y = [0.0, 0.32142412, 0.42131231, 0.66741, 0.236612, 
     1.1021, -0.32199772, -0.098712345, -0.8796421, 0.7162311]
n = 10

m = forward_backward(x, y, n)
for point in test_x:
    y_test = result(point, m, x, y, n)

### 11. Derive formula of Cubic Spline method error [Mathematical formulas]

$|| f^{(p)}(x)-S_3^{(p)}(x)||_{C[a,b]} = \max_{[a,b]}|f^{(p)}(x)-S_3^{(p)}(x)|\leq M_4h^{4-p}$, $p=0,1,2$

where $M_4=\max_{[a;b]}|f^{(4)}(x)|, h_{max}=\max_{1<=i<=n}h_i$

Note: estimation is true if $S_3(x)$ converges to $f(x) \in C_4[a,b]$

### 12. Rate the complexity of the algorithm [Text, and rate in terms of big O, no more than 100 characters]

Forward pass finds $P$ and $Q$ $n$ times, backward pass finds $n$ times too. Complexity: $\mathcal{O}(2n) = \mathcal{O}(n) $







### Congrats!